<a href="https://colab.research.google.com/github/jmrichardson/tuneta/blob/main/notebooks/tune_market.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Packages

In [ ]:
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
!pip install Ta-Lib
!pip install -U git+https://github.com/jmrichardson/tuneta

Import packages

In [1]:
from tuneta.tune_ta import TuneTA
import pandas as pd
from pandas_ta import percent_return
from sklearn.model_selection import train_test_split
import yfinance as yf
import joblib

Create data set

In [3]:
aapl = yf.download("AAPL", period="10y", interval="1d", auto_adjust=True)
aapl['sym'] = "AAPL"
aapl.set_index('sym', append=True, inplace=True)
aapl['return'] = percent_return(aapl.Close, offset=-1)

msft = yf.download("MSFT", period="10y", interval="1d", auto_adjust=True)
msft['sym'] = "MSFT"
msft.set_index('sym', append=True, inplace=True)
msft['return'] = percent_return(msft.Close, offset=-1)
   
goog = yf.download("GOOG", period="10y", interval="1d", auto_adjust=True)
goog['sym'] = "GOOG"
goog.set_index('sym', append=True, inplace=True)
goog['return'] = percent_return(goog.Close, offset=-1)

X = pd.concat([aapl, msft, goog], axis=0).sort_index()
y = X['return']
X = X.drop(columns=['return'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, shuffle=False)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Optimize

In [4]:
    # Initialize with x cores and show trial results
    tt = TuneTA(n_jobs=4, verbose=True)

    # Optimize indicators
    tt.fit(X_train, y_train,
        indicators=['tta.RSI', 'tta.MACD', 'tta.SMA', 'tta.CMO'],
        ranges=[(2, 30), (31, 60)],
        trials=300,
        early_stop=50,
    )

[I 2021-10-10 01:24:41,100] A new study created in memory with name: tta.RSI(X.close, timeperiod=trial.suggest_int('timeperiod', 2, 30), )
[I 2021-10-10 01:24:41,110] A new study created in memory with name: tta.MACD(X.close, fastperiod=trial.suggest_int('fastperiod', 2, 30), slowperiod=trial.suggest_int('slowperiod', 2, 30), signalperiod=trial.suggest_int('signalperiod', 2, 30), )
[I 2021-10-10 01:24:41,157] A new study created in memory with name: tta.SMA(X.close, timeperiod=trial.suggest_int('timeperiod', 2, 30), )
[I 2021-10-10 01:24:41,183] A new study created in memory with name: tta.CMO(X.close, timeperiod=trial.suggest_int('timeperiod', 2, 30), )
[I 2021-10-10 01:24:41,364] Trial 0 finished with value: 0.07026583431232623 and parameters: {'fastperiod': 22, 'slowperiod': 10, 'signalperiod': 8}. Best is trial 0 with value: 0.07026583431232623.
[I 2021-10-10 01:24:41,400] Trial 0 finished with value: 0.10671877442495127 and parameters: {'timeperiod': 22}. Best is trial 0 with valu

Show time duration for each indicator

In [5]:
tt.fit_times()


    Indicator      Times
--  -----------  -------
 4  tta.MACD       51.36
 5  tta.MACD       35.81
 6  tta.SMA        25.49
 7  tta.SMA        20.99
 3  tta.RSI        20.34
 1  tta.CMO        20.09
 0  tta.RSI        18.97
 2  tta.CMO        16.44


Show correlation report


In [6]:
tt.report(target_corr=True, features_corr=True)


Indicator Correlation to Target:

                                                        Correlation
----------------------------------------------------  -------------
tta_RSI_timeperiod_19                                      0.107789
tta_CMO_timeperiod_19                                      0.107789
tta_CMO_timeperiod_39                                      0.098015
tta_RSI_timeperiod_39                                      0.098015
tta_MACD_fastperiod_12_slowperiod_11_signalperiod_21       0.07416
tta_MACD_fastperiod_35_slowperiod_32_signalperiod_47       0.046094
tta_SMA_timeperiod_4                                       0.029667
tta_SMA_timeperiod_42                                      0.027647

Indicator Correlation to Each Other:

                                                        tta_RSI_timeperiod_19    tta_CMO_timeperiod_19    tta_CMO_timeperiod_39    tta_RSI_timeperiod_39    tta_MACD_fastperiod_12_slowperiod_11_signalperiod_21    tta_MACD_fastperiod_35_slowperiod_3

Remove indicators which are highly correlated with other indicators

In [6]:
tt.prune(max_correlation=.85)

Show correlation report

In [7]:
tt.report(target_corr=True, features_corr=True)


Indicator Correlation to Target:

                                                        Correlation
----------------------------------------------------  -------------
tta_RSI_timeperiod_19                                      0.107789
tta_MACD_fastperiod_12_slowperiod_11_signalperiod_21       0.07416
tta_MACD_fastperiod_35_slowperiod_32_signalperiod_47       0.046093
tta_SMA_timeperiod_4                                       0.029667

Indicator Correlation to Each Other:

                                                        tta_RSI_timeperiod_19    tta_MACD_fastperiod_12_slowperiod_11_signalperiod_21    tta_MACD_fastperiod_35_slowperiod_32_signalperiod_47    tta_SMA_timeperiod_4
----------------------------------------------------  -----------------------  ------------------------------------------------------  ------------------------------------------------------  ----------------------
tta_RSI_timeperiod_19                                                0                     

Add features to train and test sets

In [8]:
# Add indicators to X_train
features = tt.transform(X_train)
X_train = pd.concat([X_train, features], axis=1)

# Add same indicators to X_test
features = tt.transform(X_test)
X_test = pd.concat([X_test, features], axis=1)